In [1]:
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.datasets import mnist
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

from keras.models import load_model
from keras.preprocessing import image

from google.colab import files
import numpy as np

import numpy as np
%matplotlib inline

# Подготовка данных для обучения сети

In [2]:
batch_size = 100 # Размер мини-выборки
nb_classes = 10 # Количество классов изображений
nb_epoch = 25 # Количество эпох для обучения
img_rows, img_cols = 28, 28 # Размер изображений

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


# Преобразование размерности и нормализация данных
Значение интенсивности пикселей в изображении находится в интервале [0,255]. Для наших целей их необходимо нормализовать - привести к значениям в интервале [0,1].

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

Преобразуем метки классов в удобный для обучения нейронной сети формат (one hot encoding)


In [5]:
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

#Конструируем нейронную сеть и компилируем:
loss (функция потерь) - оценка желаемого значения относительно спрогнозированного, применим categorical_crossentropy (кроссэнтропию) - функцию штрафа, которую следует использовать для задач классификации, как у нас;
optimizer (функция оптимизации) - агоритм "подгонки" внутренних параметров (весов и смещений) модели для минимизации функции потерь, optimizer="adam". Метод вычисляет индивидуальные адаптивные скорости обучения для различных параметров из оценок первого и второго моментов градиентов. Название получено из adaptive moment estimation - адаптивной оценки момента. Есть другие варианты оптимизаторов;
metrics (метрики) - используются для мониторинга процесса тренировки и тестирования, metrics=['accuracy'] значит, что мы будем вычислять в модели не только функцию штрафа, но и число правильно классифицированных примеров

In [6]:
print("Если у вас уже есть модель ее можно загрузить, если нет нажмите 'cancel unload'.\n")
mod = files.upload()

Если у вас уже есть модель ее можно загрузить, если нет нажмите 'cancel unload'.



In [ ]:
pt = !ls
if pt[0].find('my_model.h5') != -1:

  model = load_model('my_model.h5')

else:

  # Создаем последовательную модель нейронной сети
  model = Sequential()
  # Первый сверточный слой
  model.add(Conv2D(img_rows, (3, 3), padding='same',
                          input_shape=(img_rows, img_cols, 1), activation='relu'))
  # Второй сверточный слой
  model.add(Conv2D(img_rows, (3, 3), activation='relu', padding='same'))
  # Первый слой подвыборки
  model.add(MaxPooling2D(pool_size=(2, 2)))
  # Слой регуляризации Dropout
  model.add(Dropout(0.25))

  # Третий сверточный слой
  model.add(Conv2D(2 * img_rows, (3, 3), padding='same', activation='relu'))
  # Четвертый сверточный слой
  model.add(Conv2D(2 * img_rows, (3, 3), activation='relu'))
  # Второй слой подвыборки
  model.add(MaxPooling2D(pool_size=(2, 2)))
  # Слой регуляризации Dropout
  model.add(Dropout(0.25))
  # Слой преобразования данных из 2D представления в плоское
  model.add(Flatten())
  # Полносвязный слой для классификации
  model.add(Dense(8 * img_rows, activation='relu'))
  # Слой регуляризации Dropout
  model.add(Dropout(0.5))
  # Выходной полносвязный слой
  model.add(Dense(nb_classes, activation='softmax'))

  #________________________________________________
  model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  print(model.summary())

  #Обучаем модель
  model.fit(x_train,y_train,epochs=5)
  #Сохраняем модель
  model.save('my_model.h5')
  #Для удаления старой модели:
  #del model
  #Для загрузки модели:
  #model = load_model('my_model.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 28)        280       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 28)        7084      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 28)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 14, 14, 28)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 56)        14168     
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 56)        28280     
                                                        

In [ ]:
#Простая оценка
model.evaluate(x_test,  y_test, verbose=2)